In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./datas/digit recognizor.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# 方差过滤




In [4]:
X = data.iloc[:,1:]
y = data.iloc[:,0]
X.shape

(42000, 784)

In [5]:
from sklearn.feature_selection import VarianceThreshold
#删除方差为0的数据
selector = VarianceThreshold() #实例化，不填参数默认方差为0
X_var0 = selector.fit_transform(X) #获取删除不合格特征之后的新特征矩阵

#也可以直接写成 X = VairanceThreshold().fit_transform(X)
X_var0.shape

(42000, 708)

In [6]:
#留下一半特征
X_fsvar = VarianceThreshold(np.median(X.var().values)).fit_transform(X) #通过中位数完成
X_fsvar.shape

(42000, 392)

In [7]:
#若特征是伯努利随机变量，假设p=0.8，即二分类特征中某种分类占到80%以上的时候删除特征
X_bvar = VarianceThreshold(.8 * (1 - .8)).fit_transform(X)
X_bvar.shape

(42000, 685)

In [8]:
# 相关性过滤




In [9]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# 卡方过滤
#假设在这里我一直我需要300个特征
X_fschi = SelectKBest(chi2, k=300).fit_transform(X_fsvar, y)
print(X_fschi.shape)

print(cross_val_score(RFC(n_estimators=10,random_state=0),X_fschi,y,cv=5).mean())
print()

(42000, 300)
0.9333098667649198



In [10]:
#选取超参数k
chivalue, pvalues_chi = chi2(X_fsvar,y)

print(chivalue) #卡方值
print()
print(pvalues_chi) #p
print()
print(chivalue.shape[0] - (pvalues_chi > 0.05).sum()) #差异性大于0.05的特征个数

#X_fschi = SelectKBest(chi2, k=填写具体的k).fit_transform(X_fsvar, y)
#cross_val_score(RFC(n_estimators=10,random_state=0),X_fschi,y,cv=5).mean()

[ 945664.84392643 1244766.05139164 1554872.30384525 1834161.78305343
 1903618.94085294 1845226.62427198 1602117.23307537  708535.17489837
  974050.20513718 1188092.19961931 1319151.43467036 1397847.8836796
 1433554.26798015 1429043.15373433 1332663.17213405 1101423.25372261
  809989.56940485  519266.71772284  285681.88297156  191589.23696468
  902883.1255264  1237265.16042373 1503477.73699155 1625807.41495542
 1630206.90922916 1630597.02883804 1633456.72909664 1610816.75571229
 1483382.49543886 1256290.1574794   951236.1617682   693192.66191748
  532386.96220361  504617.38933715  575090.36046243  501025.03733245
  802341.10683194 1078344.8724406  1226540.98318702 1269945.07968831
 1221758.57688808 1146535.17810241 1080657.20185303 1079065.30979135
 1092222.70610032 1064908.45385716 1023327.00231067  974163.15420165
  918857.12860617  861439.52030749  828439.23565047  916454.89464771
  989713.58229958  543695.5016699   674691.76755044  708113.57226969
  657819.3908855   599159.21961671 

In [11]:
from sklearn.feature_selection import f_classif

#K过滤
F, pvalues_f = f_classif(X_fsvar,y)

print(F) #F值
print()
print(pvalues_f) #p值
print()
print(F.shape[0] - (pvalues_f > 0.05).sum()) #k值大于0.05的特征个数

#X_fsF = SelectKBest(f_classif, k=填写具体的k).fit_transform(X_fsvar, y)
#cross_val_score(RFC(n_estimators=10,random_state=0),X_fsF,y,cv=5).mean()

[ 618.65383492  846.18897012 1115.40617051 1362.3677305  1452.03355369
 1381.09095571 1138.26505266  464.29616121  660.00977785  849.66393412
 1004.7450309  1124.76177588 1200.99190762 1209.29489877 1110.4944286
  854.66183292  577.52063451  342.09729054  178.67397866  118.01145533
  612.12261014  899.40904291 1196.17528948 1424.49864852 1569.26556677
 1742.49910702 1910.98023795 1969.20520223 1731.37475948 1295.09668012
  839.15325001  531.97951763  371.82392681  336.00820537  378.93378743
  317.47025479  528.94881012  766.40792176  947.63168717 1086.0472161
 1177.72017709 1253.79641973 1344.06961068 1507.33781169 1616.50454434
 1512.25864876 1289.65180587 1051.26276412  839.48869386  680.07426932
  600.85538567  633.55772663  683.96908509  347.65867784  452.76238211
  509.16387684  515.7498157   532.86107778  594.62512658  664.18740444
  709.37133696  798.11767931  876.69849088  852.76926441  785.70173347
  802.88980095  813.2041131   760.85552527  687.94148028  642.84071735
  698.11

In [12]:
from sklearn.feature_selection import mutual_info_classif as MIC

# 互信息法
result = MIC(X_fsvar,y)

k = result.shape[0] - sum(result <= 0)
print(k)

#X_fsF = SelectKBest(f_classif, k=填写具体的k).fit_transform(X_fsvar, y)
#cross_val_score(RFC(n_estimators=10,random_state=0),X_fsF,y,cv=5).mean()

392
